In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2


%matplotlib inline

print("OK\n")

OK



## 共通関数定義

In [3]:
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}




def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=1000
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [4]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成

In [5]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 234)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [6]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [7]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [9]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1168 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1122 features are remained after removing non-informative features
123 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 999 features are remained after removing features not interesting for LightGBM classifier
---=> 58 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1057 features

---=> df final shape: (356250, 1057)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Approved_VAR_low_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_high_risk,CC_NAME_CONTRACT_STATUS_Refused_VAR_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,0,0,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,0,0,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,0,0,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,1,0,0,0,1,0,1,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,0,0,0,0,0,0,0


In [12]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1057 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1051 features are remained after removing non-informative features
40 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 1011 features are remained after removing features not interesting for LightGBM classifier
---=> 19 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1030 features

---=> df final shape: (356250, 1030)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


In [13]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,0,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,0,0,1,1,0,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,0,0,1,1,0,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,1,0,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,1,1,0,0,0,0,0


## 学習

### 001 baseline

In [14]:
# split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1030)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_high_risk,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,0,0,1,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,0,0,1,1,0,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,0,1,1,0,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,0,0,1,1,0,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [15]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [16]:
app_train = app_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,ACTIVE_BUREAU_IS_DPD_OVER120_SUM_low_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,0,1,1,0,0,0,0,0,0.051055
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,0,1,1,0,0,0,0,0,0.166067
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,0,1,1,0,1,0,1,0,0.043899
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,0,1,1,0,1,0,1,0,0.056542
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,1,0,0,0,0,0,0.183168


In [17]:
#pred_train_1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_1a.csv')
#pred_test_1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1a.csv')
#pred_test_1b.head()

In [18]:
#app_train = app_train.merge(pred_train_1a, how='left', on='SK_ID_CURR')
#app_test = app_test.merge(pred_test_1a, how='left', on='SK_ID_CURR')
#app_test.head()

In [19]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.029195
1,100005,0.101040
2,100013,0.031550
3,100028,0.040153
4,100038,0.115963


In [20]:
app_train = app_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,1,0,0,0,0,0,0.051055,0.029195
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,1,0,0,0,0,0,0.166067,0.101040
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,1,0,1,0,1,0,0.043899,0.031550
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,1,0,1,0,1,0,0.056542,0.040153
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,1,0,0,0,0,0,0.183168,0.115963


In [21]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.038205
1,100005,0.103087
2,100013,0.058075
3,100028,0.035603
4,100038,0.184927


In [22]:
app_train = app_train.merge(train_set3a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3a, how='left', on='SK_ID_CURR')
app_test.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.051055,0.029195,0.038205
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.166067,0.101040,0.103087
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.043899,0.031550,0.058075
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.056542,0.040153,0.035603
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.183168,0.115963,0.184927


In [23]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,1,1,0,0,0,0,0,0.051055,0.029195,0.038205
1,1,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,1,1,0,0,0,0,0,0.166067,0.101040,0.103087
2,2,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,1,1,0,1,0,1,0,0.043899,0.031550,0.058075
3,3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,1,1,0,1,0,1,0,0.056542,0.040153,0.035603
4,4,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0,1,0,0,0,0,0,0.183168,0.115963,0.184927


In [24]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.051055,0.029195,0.038205
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.166067,0.101040,0.103087
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.043899,0.031550,0.058075
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.056542,0.040153,0.035603
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.183168,0.115963,0.184927


### 正規化＋対数変換

In [25]:
app_train_ord = app_train.copy()
app_test_ord = app_test.copy()
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.051055,0.029195,0.038205
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.166067,0.101040,0.103087
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.043899,0.031550,0.058075
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.056542,0.040153,0.035603
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.183168,0.115963,0.184927


In [26]:
# infをnanへ置換
app_train_ord = app_train_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord = app_test_ord.replace([np.inf, -np.inf], np.nan)
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.051055,0.029195,0.038205
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.166067,0.101040,0.103087
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.043899,0.031550,0.058075
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.056542,0.040153,0.035603
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.183168,0.115963,0.184927


In [27]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(app_test_ord.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [28]:
### 欠損値補完
#app_train_ord = app_train_ord.fillna(app_train_ord.max() + (app_train_ord.max() * 0.4))
app_train_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1,1,0,0,0,0,0,0.516856,0.177023,0.254294
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,1,1,0,0,0,0,0,0.010645,0.012354,0.022026
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,1,1,0,0,0,0,0,0.054077,0.040684,0.035999
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,1,0,1,0,0,0,0.059492,0.030685,0.021632
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1,1,0,0,0,0,0,0.076165,0.041810,0.025333


In [29]:
### 欠損値補完
#app_test_ord = app_test_ord.fillna(app_test_ord.max() + (app_test_ord.max() * 0.4))
app_test_ord.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,1,1,0,0,0,0,0,0.051055,0.029195,0.038205
1,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,1,1,0,0,0,0,0,0.166067,0.101040,0.103087
2,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,1,1,0,1,0,1,0,0.043899,0.031550,0.058075
3,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,1,1,0,1,0,1,0,0.056542,0.040153,0.035603
4,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,1,0,0,0,0,0,0.183168,0.115963,0.184927


In [30]:
### Yao-Jhonson変換
#for column in encoding_columns:
#    modules.process_yj_convert(app_train_ord, app_test_ord, column, [column])#
#
#app_test_ord.head()

In [31]:
### indexを設定
app_train_ord = app_train_ord.set_index('SK_ID_CURR')
app_test_ord = app_test_ord.set_index('SK_ID_CURR')
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0,NaN,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,1,1,0,0,0,0,0,0.051055,0.029195,0.038205
100005,1,NaN,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,1,1,0,0,0,0,0,0.166067,0.101040,0.103087
100013,2,NaN,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,1,0,1,0,0.043899,0.031550,0.058075
100028,3,NaN,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,1,0,1,0,0.056542,0.040153,0.035603
100038,4,NaN,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,0,0,0,0,0.183168,0.115963,0.184927


In [32]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train_ord, app_test_ord, num_cols)
        
app_test_ord.head()

,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,...,0.579736,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,-0.369428,-0.475536,-0.436774
100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,...,0.579736,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,0.969472,0.289877,0.236697
100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,0.247943,...,0.579736,0.245284,0.0,1.685055,-0.150513,2.007802,0.0,-0.452737,-0.450452,-0.230531
100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,2.805819,...,0.579736,0.245284,0.0,1.685055,-0.150513,2.007802,0.0,-0.305554,-0.358800,-0.463784
100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,0.235763,...,-1.724923,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,1.168560,0.448853,1.086186


In [33]:
### indexを削除
app_train_ord = app_train_ord.reset_index()
app_test_ord = app_test_ord.reset_index()
app_test_ord.head()

,SK_ID_CURR,level_0,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,PREV_PREV_GOODS_DIFF_MAX_high_risk,PREV_PREV_GOODS_DIFF_MAX_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,pred_1b,set1,set3a
0,100001,-1.732045,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,...,0.579736,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,-0.369428,-0.475536,-0.436774
1,100005,-1.732034,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,...,0.579736,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,0.969472,0.289877,0.236697
2,100013,-1.732023,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,...,0.579736,0.245284,0.0,1.685055,-0.150513,2.007802,0.0,-0.452737,-0.450452,-0.230531
3,100028,-1.732011,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,...,0.579736,0.245284,0.0,1.685055,-0.150513,2.007802,0.0,-0.305554,-0.358800,-0.463784
4,100038,-1.732000,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,...,-1.724923,0.245284,0.0,-0.593453,-0.150513,-0.498057,0.0,1.168560,0.448853,1.086186


In [34]:
x_train = app_train_ord.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train_ord['TARGET']
id_train = app_train_ord[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [35]:
# データセット作成
x_test = app_test_ord.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test_ord[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [36]:
#del app_train_ord
#del app_test_ord
#gc.collect()

In [37]:
#for col in app_train_ord.columns:
#    print(col)

In [38]:
### パラメータ
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 2000
}


### 学習モデル構築
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 1031) (61502, 1031)
[100]	training's auc: 0.792196	valid_1's auc: 0.779404
[200]	training's auc: 0.79957	valid_1's auc: 0.78335
[300]	training's auc: 0.805915	valid_1's auc: 0.786864
[400]	training's auc: 0.811102	valid_1's auc: 0.788869
[500]	training's auc: 0.81588	valid_1's auc: 0.790149
[600]	training's auc: 0.820237	valid_1's auc: 0.791085
[700]	training's auc: 0.824374	valid_1's auc: 0.791808
[800]	training's auc: 0.828079	valid_1's auc: 0.792316
[900]	training's auc: 0.831727	valid_1's auc: 0.792583
[1000]	training's auc: 0.835229	valid_1's auc: 0.792832
[1100]	training's auc: 0.838494	valid_1's auc: 0.792989
[1200]	training's auc: 0.84176	valid_1's auc: 0.793116
[1300]	training's auc: 0.844774	valid_1's auc: 0.79328
[1400]	training's auc: 0.847618	valid_1's auc: 0.793509
[1500]	training's auc: 0.850423	valid_1's auc: 0.793537
[1600]	training's auc: 0.853158	valid_1's auc: 0.793483
[1700]	training's auc: 0.855843	valid_1's auc

In [39]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
1028,pred_1b,862.4,19.882153
727,PAYMENT_RATE,710.0,55.475220
1030,set3a,482.2,14.498276
1029,set1,440.8,18.899735
453,DAYS_BIRTH,317.6,4.393177
65,AMT_GOODS_PRICE,247.6,25.735190
140,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff,233.6,17.657859
759,POS_REMAINING_INSTALMENTS,218.6,22.711231
63,AMT_ANNUITY,215.0,13.638182
483,EXT_SOURCE_3,209.8,20.265735


In [40]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [41]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_featureEngineeringOtherSource2-2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.039482
1,100005,0.096614
2,100013,0.043337
3,100028,0.044793
4,100038,0.167878


## パラメータチューニング

In [42]:
#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [43]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [44]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [45]:
#train_oof, imp, metrics = baseline.train_lgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [46]:
#app_test_ord

In [47]:
### 推論用データセットの作成
#x_test = app_test_ord.drop(columns=['SK_ID_CURR','TARGET'])
#id_test = app_test_ord[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

### predict
#test_pred = baseline.predict_lgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

### make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_HyperParameterTuningLgb.csv", index=None)

In [48]:
#del app_train_ord
#del app_test_ord
#gc.collect()